In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from collections import Counter
import holidays
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output


In [3]:
#### read files
train_identity = pd.read_csv("/Users/s0c02nj/Desktop/Kaggle/train_identity.csv")
train_transaction = pd.read_csv("/Users/s0c02nj/Desktop/Kaggle/train_transaction.csv")
test_transaction = pd.read_csv("/Users/s0c02nj/Desktop/Kaggle/test_transaction.csv")
test_identity = pd.read_csv('/Users/s0c02nj/Desktop/Kaggle/test_identity.csv')

In [11]:
train_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [33]:
len(set(test_identity['TransactionID']).intersection(set(train_transaction['TransactionID'])))

0

In [36]:
#test_identity.shape

In [37]:
train_transaction['isFraud'].value_counts()

0    569877
1     20663
Name: isFraud, dtype: int64

In [41]:
train_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
test_transaction['P_emaildomain'].unique()

array(['gmail.com', 'aol.com', 'hotmail.com', nan, 'att.net', 'twc.com',
       'yahoo.com', 'verizon.net', 'anonymous.com', 'msn.com', 'q.com',
       'outlook.com', 'icloud.com', 'bellsouth.net', 'sbcglobal.net',
       'web.de', 'live.com.mx', 'ymail.com', 'comcast.net', 'mail.com',
       'cox.net', 'me.com', 'aim.com', 'charter.net', 'rocketmail.com',
       'outlook.es', 'centurylink.net', 'live.com', 'yahoo.de',
       'cfl.rr.com', 'mac.com', 'frontier.com', 'netzero.net',
       'frontiernet.net', 'juno.com', 'windstream.net', 'yahoo.fr',
       'netzero.com', 'optonline.net', 'yahoo.co.uk', 'cableone.net',
       'roadrunner.com', 'sc.rr.com', 'earthlink.net', 'gmail',
       'hotmail.de', 'yahoo.com.mx', 'suddenlink.net', 'yahoo.es',
       'embarqmail.com', 'hotmail.es', 'ptd.net', 'hotmail.co.uk',
       'protonmail.com', 'live.fr', 'prodigy.net.mx', 'yahoo.co.jp',
       'hotmail.fr', 'gmx.de', 'servicios-ta.com', 'scranton.edu'],
      dtype=object)

In [64]:
#list(test_transaction.columns)


In [61]:
from sklearn.decomposition import PCA

In [62]:
pca = PCA(n_components=2)

In [67]:
cols = []
for i in range(1,340):
    cols.append('V'+str(i))

In [112]:
df = train_transaction[cols]

In [115]:
for col in tqdm(cols):
     df[col] = df[col].replace(np.nan,df[col].mode()[0])

  0%|          | 0/339 [00:00<?, ?it/s]/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
100%|██████████| 339/339 [00:28<00:00, 11.93it/s]


In [117]:
pca.fit(df)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [119]:
pca.explained_variance_ratio_

array([0.92920758, 0.0642376 ])

In [121]:
test_transaction.shape

(506691, 393)

In [126]:
train_transaction = train_transaction.sort_values(by=['TransactionDT'],ascending = True)
test_transaction = test_transaction.sort_values(by=['TransactionDT'],ascending = True)

In [128]:
train_transaction.tail()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590539,3577539,0,15811131,279.95,W,15066,170.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
test_transaction.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
ind1 = train_transaction.shape[0] * 0.6
ind2 = train_transaction.shape[0] * 0.4

df_t1 = train_transaction.loc[0:ind1]
df_t2 = train_transaction.loc[ind1:]

In [148]:
df_t1['isFraud'].value_counts()

0    342337
1     11988
Name: isFraud, dtype: int64

In [149]:
df_t2['isFraud'].value_counts()

0    227541
1      8675
Name: isFraud, dtype: int64

In [150]:
11988/(342337 + 11988) 

0.033833345092782054

In [146]:
4064/(114044 + 4064)

0.034409184813899145